# Data Loading

In [ ]:
#import necessary libraries

import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('datasets copies/DispatchSummary.csv')

In [ ]:
data.columns

In [ ]:
data.columns=['financial_year', 'sales', 'man_power', 'available_hours', 'inc_smh', 'efficiency', 'smh', 'booked', 'smh_lacs']

In [ ]:
data.columns

In [ ]:
dataDic={}
dataDic['financial_year']=data['financial_year'][0:9]
dataDic['sales']=data['sales'][0:9]
dataDic['man_power']=data['man_power'][0:9]
dataDic['efficiency']=data['efficiency'][0:9]
dataDic['booked']=data['booked'][0:9]
dataDic['dispatch_smh']=data['inc_smh'][0:9]
dataDic['inhouse_smh']=data['smh'][0:9]
dataDic['outsourced_smh']=data['smh_lacs'][0:9]

print(dataDic)

In [ ]:
df=pd.DataFrame(dataDic)
print(df)

# Efficiency prediction

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Create and fit the ARIMA model
model = ARIMA(data['efficiency'][0:9], order=(1, 1, 1)) # Adjust (p, d, q) based on you
model_fit = model.fit()
# Forecast the next 5 periods (5 years in this case)
forecast = model_fit.forecast(steps=6)
# Print the forecasted values
print(forecast)


In [ ]:
import pickle

pickle.dump(model_fit, open('effModel.pkl', 'wb'))

In [ ]:
from decimal import Decimal, ROUND_DOWN

for i in range(6):
    value=Decimal(forecast[i+9])
    forecast[i+9]=value.quantize(Decimal('0.01'), rounding=ROUND_DOWN)
print(forecast)

In [ ]:
fin_year=data['financial_year']
print(len(fin_year))
efficiency_ar=pd.concat([df['efficiency'], pd.DataFrame(forecast)['predicted_mean']], axis=0)
efficiency_ar=efficiency_ar.reset_index()

#display
print('Financial year\tEfficiency')
for i in range(len(fin_year)):
    print(fin_year[i],'\t',efficiency_ar[0][i])

#plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(fin_year, efficiency_ar[0], marker='o')
plt.xlabel('Financial year')
plt.ylabel('Efficiency')
plt.title('Efficiency over years')
plt.grid()
plt.savefig('effOverYears.png')
plt.show()

# Inhouse smh

In [ ]:
booked_test=data['booked'][9:15].reset_index()
financial_year_test=data['financial_year'][9:15].reset_index()

fut_efficiency=forecast.reset_index()
inhouse_smh_predicted = (booked_test['booked'] * fut_efficiency['predicted_mean']) / 100
inhouse_smh_predicted = inhouse_smh_predicted.reset_index()
inhouse_smh_predicted.columns=['index', 'smh']

In [ ]:
from decimal import Decimal, ROUND_DOWN

for i in range(6):
    value=Decimal(inhouse_smh_predicted['smh'][i])
    inhouse_smh_predicted['smh'][i]=value.quantize(Decimal('0.01'), rounding=ROUND_DOWN)
print(inhouse_smh_predicted['smh'])

In [ ]:
print('Prediction on future data')
print('Financial year\tEfficiency\tInhouse SMH\t\tBooked')
for i in range(len(booked_test)):
    print(financial_year_test['financial_year'][i],'\t',fut_efficiency['predicted_mean'][i],'\t',inhouse_smh_predicted['smh'][i],'\t',booked_test['booked'][i])


**Overall Data**

In [ ]:
manpower=data['man_power']

inhouse_smh=pd.concat([pd.DataFrame({'smh':df['inhouse_smh']}), pd.DataFrame(inhouse_smh_predicted['smh'])], axis=0)
inhouse_smh=inhouse_smh.reset_index()

print('Financial year\tMan Power\tEfficiency\tInnhouse SMH')
for i in range(len(manpower)):
    print(fin_year[i], '\t\t', manpower[i], '\t\t', efficiency_ar[0][i], '\t\t\t', inhouse_smh['smh'][i], sep='')

In [ ]:
#plot(financial year vs inhouse smh)

plt.figure(figsize=(12, 6))
plt.plot(fin_year, inhouse_smh['smh'], marker='o')
plt.xlabel('Financial Year')
plt.ylabel('Inhouse SMH')
plt.title('Inhouse SMH over years')
plt.grid()
plt.savefig('inhouseOverYears.png')
plt.show()

# Dispatch smh

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Create and fit the ARIMA model
dispatch_model = ARIMA(df['dispatch_smh'], order=(2, 3, 1)) # Adjust (p, d, q) based on you
dispatch_model_fit = dispatch_model.fit()
# Forecast the next 5 periods (5 years in this case)
dispatch_forecast = dispatch_model_fit.forecast(steps=6)
# Print the forecasted values
print(dispatch_forecast)


In [ ]:
import pickle

pickle.dump(dispatch_model_fit, open('dispatchModel.pkl', 'wb'))

In [ ]:
from decimal import Decimal, ROUND_DOWN

for i in range(6):
    value=Decimal(dispatch_forecast[i+9])
    dispatch_forecast[i+9]=value.quantize(Decimal('0.01'), rounding=ROUND_DOWN)
print(dispatch_forecast)

In [ ]:
fin_year=data['financial_year']
dispatch_ar=pd.concat([df['dispatch_smh'], pd.DataFrame(dispatch_forecast)['predicted_mean']], axis=0)
dispatch_ar=dispatch_ar.reset_index()

#display
print('Financial year\tDispatch SMH')
for i in range(len(fin_year)):
    print(fin_year[i],'\t',dispatch_ar[0][i])

#plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(fin_year, dispatch_ar[0], marker='o')
plt.xlabel('Financial year')
plt.ylabel('Dispatch SMH')
plt.title('Dispatch SMH over years')
plt.grid()
plt.savefig('dispatchOverYears.png')
plt.show()

In [ ]:
import itertools
p=d=q=range(0,5)
pdq=list(itertools.product(p,d,q))
param_aic={}
for param in pdq:
    try:
        model=ARIMA(data['inc_smh'],order=param)
        model_fit=model.fit()
        param_aic[param]=model_fit.aic
    except:
        continue

In [ ]:
min_key = min(param_aic, key=param_aic.get)
min_value = param_aic[min_key]
print(min_key, '->', min_value)

# Outsourced SMH

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Create and fit the ARIMA model
outsourced_model = ARIMA(df['outsourced_smh'], order=(2, 1, 1)) # Adjust (p, d, q) based on you
outsourced_model_fit = outsourced_model.fit()
# Forecast the next 5 periods (5 years in this case)
outsourced_forecast =outsourced_model_fit.forecast(steps=6)
# Print the forecasted values
print(outsourced_forecast)


In [ ]:
import pickle

pickle.dump(outsourced_model_fit, open('outsourcedModel.pkl', 'wb'))

In [ ]:
from decimal import Decimal, ROUND_DOWN

for i in range(6):
    value=Decimal(outsourced_forecast[i+9])
    outsourced_forecast[i+9]=value.quantize(Decimal('0.01'), rounding=ROUND_DOWN)
print(outsourced_forecast)

In [ ]:
fin_year=data['financial_year']
outsourced_ar=pd.concat([df['outsourced_smh'], pd.DataFrame(outsourced_forecast)['predicted_mean']], axis=0)
outsourced_ar=outsourced_ar.reset_index()

#display
print('Financial year\tOutsourced SMH')
for i in range(len(fin_year)):
    print(fin_year[i],'\t',outsourced_ar[0][i])

#plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(fin_year, outsourced_ar[0], marker='o')
plt.xlabel('Financial year')
plt.ylabel('outsourced SMH')
plt.title('outsourced SMH over years')
plt.grid()
plt.savefig('outsourcedOverYears.png')
plt.show()

In [ ]:
import itertools
p=d=q=range(0,5)
pdq=list(itertools.product(p,d,q))
param_aic={}
for param in pdq:
    try:
        model=ARIMA(data['smh_lacs'],order=param)
        model_fit=model.fit()
        param_aic[param]=model_fit.aic
    except:
        continue

In [ ]:
min_key = min(param_aic, key=param_aic.get)
min_value = param_aic[min_key]
print(min_key, '->', min_value)

In [ ]:
from statsmodels.tsa.stattools import adfuller
outsorced_smh=data['smh_lacs'][0:9]
result = adfuller(outsorced_smh)
d=0
while result[1] > 0.05:
    d+=1
    outsorced_smh = outsorced_smh.diff().dropna()
    result = adfuller(outsorced_smh.dropna())
print('Value of d: ', d)

In [ ]:
man_power=data['man_power']
print('Financial Year\tMan Power\tEfficiency\tInhouse SMH\tDispatch SMH\tOutsourced SMH')
for i in range(len(man_power)):
    print(fin_year[i],'\t',man_power[i],'\t',efficiency_ar[0][i],'\t',inhouse_smh['smh'][i],'\t',dispatch_ar[0][i],'\t',outsourced_ar[0][i])



In [ ]:
predicted=pd.DataFrame({
    'financial_year' : data['financial_year'][9:15].reset_index()['financial_year'],
    'sales' : data['sales'][9:15].reset_index()['sales'],
    'man_power': data['man_power'][9:15].reset_index()['man_power'],
    'efficiency' : fut_efficiency['predicted_mean'],
    'dispatch_smh' : dispatch_forecast.reset_index()['predicted_mean'],
    'inhouse_smh' : inhouse_smh_predicted['smh'],
    'outsourced_smh' : outsourced_forecast.reset_index()['predicted_mean']

})

print(predicted)

In [ ]:
import pickle

#dump
pickle.dump(predicted, open('predictedValues.pkl', 'wb'))